## Membres du Groupe de Projet

- Maxence KAMIONKA
- Mikhaïl BENALI
- Hadja BAH
- Emmanuel DAGNOGO


In [1]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


Using device: cuda


In [2]:
import gc

# Libérer la RAM Python
gc.collect()
# Libérer la mémoire GPU inutilisée
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Test du modèle de base

In [3]:
from transformers import AutoTokenizer

MODEL_NAME = "t5-base"
INSTRUCTION = "translate English to French: "  

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoModelForSeq2SeqLM

base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)


## Utilisation de LoRa (Low-Rank Adaption)

On utilise LoRA afin d'entrainer des paramètres extra tout en gelant ceux du modèle de base pour ne pas lui faire oublier ce qu'il sait déjà faire originellement. On tune alors ces poids supplémentaires plutôt que tout le modèle.

In [5]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, TaskType, get_peft_model

def load_lora_model():
    # on réutilise MODEL_NAME & tokenizer définis plus haut
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

    model.config.no_repeat_ngram_size = 3

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        inference_mode=False,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
    )

    model = get_peft_model(model, peft_config)
    model.to(DEVICE)
    model.print_trainable_parameters()

    return model


### Choix des jeux de données

OPUS avait été considéré plus tôt mais les traductions un peu "étranges" remarquées ont fait que le dataset Europarl a été privilégié à la fin.


**Europarl en-fr** : phrases longues, style plus formel, discours parlementaires -> plus de contexte par phrase.


**Ajouter les TED Talks plus tard pour le langage oral, plus “conversational”, avec un ton explicatif.**

### Charger et échantilloner le dataset OPUS et Europarl

In [6]:
from datasets import load_dataset, concatenate_datasets

def load_data_old(sample_size_train=80000, sample_size_val=1000):

    ### OPUS ###
    opus = load_dataset("Helsinki-NLP/opus-100", "en-fr")
    opus_train = opus["train"].shuffle(seed=42).select(range(sample_size_train))
    opus_val = opus["validation"].shuffle(seed=42).select(range(sample_size_val))

    ### EUROPARL EN-FR ###
    europarl = load_dataset("Helsinki-NLP/europarl", "en-fr", split="train").shuffle(seed=42)
    euro_train = europarl.select(range(sample_size_train))
    euro_val = europarl.select(range(sample_size_train, sample_size_train + sample_size_val))

    ### FUSION des 2 datasets ###
    train_ds = concatenate_datasets([opus_train, euro_train])
    val_ds = concatenate_datasets([opus_val, euro_val])
    
    # re-shuffle global
    train_ds = train_ds.shuffle(seed=43)
    val_ds   = val_ds.shuffle(seed=43)
    
    return train_ds, val_ds


def load_data(sample_size_train=5000, sample_size_val=200):

    ### EUROPARL EN-FR ###
    europarl = load_dataset("Helsinki-NLP/europarl", "en-fr", split="train").shuffle(seed=42)
    euro_train = europarl.select(range(sample_size_train))
    euro_val = europarl.select(range(sample_size_train, sample_size_train + sample_size_val))
    
    return euro_train, euro_val


In [7]:
# Charger les données brutes
train_ds, val_ds = load_data()

def show_raw_example(ds, idx=0, prefix="train"):
    ex = ds[idx]
    print(f"--- {prefix} example {idx} ---")
    print("EN :", ex["translation"]["en"])
    print("FR :", ex["translation"]["fr"])
    print()

# 3 exemples du train et 3 de la val
for i in range(3):
    show_raw_example(train_ds, i, prefix="train")

for i in range(3):
    show_raw_example(val_ds, i, prefix="val")


--- train example 0 ---
EN : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
FR : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.

--- train example 1 ---
EN : It does indeed speak for itself that those who are around the negotiating table are most sensitive to their own issues; this is always the case.
FR : Il est d'ailleurs évident que les personnes qui siègent autour de la table de négociations sont les plus sensibles à leurs propres problèmes. Il en est toujours ainsi.

--- train example 2 ---
EN : We nevertheless believe that the compromise is sound overall because it constitutes a clear improvement upon the original proposal.
FR : Cependant, nous estimons que le compromis est dans l’ensemble satisfaisant, en ce sens qu’il 

### Encoder l’anglais comme input et le français comme labels

In [8]:
import numpy as np

def preprocess_function(examples, tokenizer):
    inputs = [INSTRUCTION + ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    # Encodage des entrées
    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
    )

    # Encodage des cibles
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation=True,
            padding="max_length",
        )

    label_ids = labels["input_ids"]
    pad_token_id = tokenizer.pad_token_id

    # ignorer les PAD dans la loss
    label_ids = [
        [(tok if tok != pad_token_id else -100) for tok in seq]
        for seq in label_ids
    ]

    model_inputs["labels"] = label_ids
    return model_inputs


In [9]:
def tokenize_datasets(train, val, tokenizer):
    tokenized_train = train.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=train.column_names,
    )
    tokenized_val = val.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=val.column_names,
    )
    return tokenized_train, tokenized_val


In [10]:
# Charger modèle LoRA & réutiliser le tokenizer global
model = load_lora_model()

train_ds, val_ds = load_data()
tokenized_train, tokenized_val = tokenize_datasets(train_ds, val_ds, tokenizer)



trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953


Map:   0%|                                                                              | 0/200 [00:00<?, ? examples/s]C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 4271.42 examples/s]


In [11]:
def show_processed_example(raw_ds, tokenized_ds, idx=0, prefix="train"):
    raw = raw_ds[idx]
    tok = tokenized_ds[idx]

    print(f"=== {prefix} example {idx} ===")
    print("RAW EN :", raw["translation"]["en"])
    print("RAW FR :", raw["translation"]["fr"])
    print()

    # Inputs
    print("input_ids[:20] :", tok["input_ids"][:20])
    print("Decoded input  :", tokenizer.decode(tok["input_ids"], skip_special_tokens=True))
    print()

    # Labels (enlevant les -100 pour re-décoder)
    labels = tok["labels"]
    # On remplace les -100 par pad_token_id pour pouvoir décoder
    pad_id = tokenizer.pad_token_id
    labels_for_decode = [pad_id if x == -100 else x for x in labels]
    print("labels[:20]     :", labels[:20])
    print("Decoded labels  :", tokenizer.decode(labels_for_decode, skip_special_tokens=True))
    print()


In [12]:
for i in range(3):
    show_processed_example(train_ds, tokenized_train, i, prefix="train")

for i in range(3):
    show_processed_example(val_ds, tokenized_val, i, prefix="val")


=== train example 0 ===
RAW EN : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
RAW FR : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.

input_ids[:20] : [13959, 1566, 12, 2379, 10, 282, 1611, 7, 6, 28, 69, 351, 6, 69, 1543, 13, 3065, 11, 69, 1456]
Decoded input  : translate English to French: As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.

labels[:20]     : [622, 2430, 3890, 35, 7, 238, 678, 7056, 6, 393, 1089, 11183, 6, 1089, 1543, 20, 50, 25060, 3, 15]
Decoded labels  : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur 

### Métrique d'évaluation de la traduction

In [13]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

## Pour nettoyer le texte
def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels


def compute_metrics(eval_preds, tokenizer):
    # eval_preds peut être un tuple (preds, labels)
    # ou un objet EvalPrediction avec .predictions et .label_ids
    if hasattr(eval_preds, "predictions"):
        preds = eval_preds.predictions
        labels = eval_preds.label_ids
    else:
        preds, labels = eval_preds

    # Certains modèles renvoient (logits, ...) -> on garde seulement le 1er élément
    if isinstance(preds, tuple):
        preds = preds[0]

    # On met tout en np.array pour être tranquille
    preds = np.array(preds)

    # Cas où preds = logits (batch, seq_len, vocab_size) -> on prend l'argmax
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # On s'assure que ce sont bien des entiers
    preds = preds.astype("int64")

    # si jamais il y a des valeurs négatives dans preds on les remplace par pad_token_id avant decode
    preds[preds < 0] = tokenizer.pad_token_id

    # Gestion des labels : on remet pad_token_id à la place des -100 pour décoder
    labels = np.array(labels)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode des prédictions et des labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage simple
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # sacreBLEU / METEOR / ROUGE
    # sacreBLEU attend une liste de listes pour les références
    refs_list = [[r] for r in decoded_labels]

    bleu_res = sacrebleu.compute(
        predictions=decoded_preds,
        references=refs_list,
    )

    rouge_res = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )

    meteor_res = meteor.compute(
        predictions=decoded_preds,
        references=decoded_labels,
    )

    return {
        "bleu": bleu_res["score"],
        "meteor": meteor_res["meteor"],
        "rouge1": rouge_res["rouge1"],
        "rouge2": rouge_res["rouge2"],
        "rougeL": rouge_res["rougeL"],
    }

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\maxka\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Métriques sur modèle de base

In [14]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Modèle de base (non fine-tuné)
model_base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

model_base.config.no_repeat_ngram_size = 3

# On réutilise le même val set tokenisé
data_collator_base = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_base,
)

eval_args_base = Seq2SeqTrainingArguments(
    output_dir="baseline_t5_base_en_fr",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    do_train=False,
    do_eval=True,
    logging_dir="logs_baseline",
)

trainer_base = Seq2SeqTrainer(
    model=model_base,
    args=eval_args_base,
    eval_dataset=tokenized_val,  # même val que pour LoRA
    tokenizer=tokenizer,
    data_collator=data_collator_base,
    compute_metrics=lambda p: compute_metrics(p, tokenizer),
)

baseline_metrics = trainer_base.evaluate()
print("Baseline metrics :", baseline_metrics)


C:\Users\maxka\AppData\Local\Temp\ipykernel_12832\3450106949.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_base = Seq2SeqTrainer(
C:\Users\maxka\anaconda3\envs\llm_trad\Lib\site-packages\transformers\generation\utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Baseline metrics : {'eval_loss': 1.0082529783248901, 'eval_model_preparation_time': 0.003, 'eval_bleu': 34.32950213928655, 'eval_meteor': 0.6026120330172933, 'eval_rouge1': 0.6638890688050059, 'eval_rouge2': 0.45996260002474126, 'eval_rougeL': 0.620193619313274, 'eval_runtime': 45.4089, 'eval_samples_per_second': 4.404, 'eval_steps_per_second': 0.551}


In [15]:
from transformers import TrainerCallback

class GarbageCollectorCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Libérer la RAM Python
        gc.collect()
        # Libérer la mémoire GPU inutilisée
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return control


## Trainer

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

def train(model, tokenizer, tokenized_train, tokenized_val):
    training_args = Seq2SeqTrainingArguments(
        output_dir="finetuned_t5_base_en_fr",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        predict_with_generate=True,
        load_best_model_at_end=True,
        metric_for_best_model="eval_bleu",
        greater_is_better=True,
        logging_steps=50,

        generation_max_length=128,
        generation_num_beams=4,
    )

    gc_callback = GarbageCollectorCallback()

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(p, tokenizer),
        callbacks=[gc_callback],
    )

    trainer.train()

    trainer.model.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    print("Best eval :", trainer.state.best_metric)
    print("Best checkpoint :", trainer.state.best_model_checkpoint)

    return trainer


## Entrainement du modèle fine-tuné

In [17]:
# 1) Entraîner le modèle LoRA
trainer = train(model, tokenizer, tokenized_train, tokenized_val)

# 2) Évaluer proprement le modèle LoRA sur le même jeu de validation
lora_metrics = trainer.evaluate(eval_dataset=tokenized_val)
print("Métriques modèle de base :", baseline_metrics)
print("Métriques modèle LoRA    :", lora_metrics)

C:\Users\maxka\AppData\Local\Temp\ipykernel_12832\302199612.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Bleu,Meteor,Rouge1,Rouge2,Rougel
1,0.792700,0.767098,36.737944,0.613811,0.673695,0.475615,0.631384
2,0.808600,0.764936,36.717175,0.614851,0.674162,0.474805,0.631212
3,0.737900,0.764149,36.697760,0.614943,0.673988,0.474629,0.631332


Best eval : 36.73794379438751
Best checkpoint : finetuned_t5_base_en_fr\checkpoint-625


Métriques modèle de base : {'eval_loss': 1.0082529783248901, 'eval_model_preparation_time': 0.003, 'eval_bleu': 34.32950213928655, 'eval_meteor': 0.6026120330172933, 'eval_rouge1': 0.6638890688050059, 'eval_rouge2': 0.45996260002474126, 'eval_rougeL': 0.620193619313274, 'eval_runtime': 45.4089, 'eval_samples_per_second': 4.404, 'eval_steps_per_second': 0.551}
Métriques modèle LoRA    : {'eval_loss': 0.7670983672142029, 'eval_bleu': 36.73794379438751, 'eval_meteor': 0.6138114364037918, 'eval_rouge1': 0.673695196927682, 'eval_rouge2': 0.475615149964153, 'eval_rougeL': 0.6313841125429722, 'eval_runtime': 46.3403, 'eval_samples_per_second': 4.316, 'eval_steps_per_second': 0.539, 'epoch': 3.0}


In [18]:
import pandas as pd
logs = pd.DataFrame(trainer.state.log_history)

# Lignes d'éval (celles qui ont une eval_loss)
eval_logs = logs[logs["eval_loss"].notna()]

# Colonnes qui nous intéressent
cols = ["epoch", "step", "eval_loss", "eval_bleu", "eval_meteor", "eval_rougeL"]

print(eval_logs[cols])

    epoch  step  eval_loss  eval_bleu  eval_meteor  eval_rougeL
12    1.0   625   0.767098  36.737944     0.613811     0.631384
26    2.0  1250   0.764936  36.717175     0.614851     0.631212
39    3.0  1875   0.764149  36.697760     0.614943     0.631332
41    3.0  1875   0.767098  36.737944     0.613811     0.631384


# Recharger le modèle fine-tuné

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftConfig, PeftModel

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

BASE_MODEL_NAME = "t5-base"
PEFT_DIR = "./finetuned_t5_base_en_fr"


Using device: cuda


## Charger le modèle LoRA et de base

In [20]:
# 1) Tokenizer commun (T5-base) : on le charge UNE seule fois
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# 2) Modèle de base (non fine-tuné)
base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME).to(DEVICE)
base_model.eval()

# 3) Modèle LoRA fine-tuné (chargé depuis le dossier PEFT_DIR)
lora_backbone = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
lora_model = PeftModel.from_pretrained(
    lora_backbone,
    PEFT_DIR,
    local_files_only=True,
).to(DEVICE)
lora_model.eval()

print("Modèle de base + modèle LoRA rechargés depuis les chemins.")


Modèle de base + modèle LoRA rechargés depuis les chemins.


## Fonction de traduction + tests

In [21]:
INSTRUCTION = "translate English to French: "

def translate_sentence(sentence, model, tokenizer, max_length=256, num_beams=4):
    model.eval()

    inputs = tokenizer(
        INSTRUCTION + sentence,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=num_beams,
            no_repeat_ngram_size=3,  # comme dans model_base.config
        )
        
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [22]:
# Quelques phrases du dataset train
for i in range(3):
    en = train_ds[i]["translation"]["en"]
    fr_gold = train_ds[i]["translation"]["fr"]

    fr_base = translate_sentence(en, base_model, tokenizer)
    fr_lora = translate_sentence(en, lora_model, tokenizer) # modèle fine-tuné LoRA

    print("--- Exemple train", i, "---")
    print("EN      :", en)
    print("FR gold :", fr_gold)
    print("FR base :", fr_base)
    print("FR LoRA :", fr_lora)
    print()

--- Exemple train 0 ---
EN      : As Europeans, with our experience, our culture of peace and our economic opportunities, we too are called upon to make our contribution towards a better future for Iraq.
FR gold : Les Européens que nous sommes, avec leur expérience, leur culture de la paix et leurs moyens économiques, sont appelés à apporter leur contribution en faveur d'un avenir meilleur en Irak.
FR base : En tant qu'Européens, avec notre expérience, notre culture de paix et nos possibilités économiques, nous sommes également appelés à contribuer à un avenir meilleur pour l'Iraq.
FR LoRA : En tant qu'Européens, avec notre expérience, notre culture de paix et nos perspectives économiques, nous sommes également appelés à contribuer à un meilleur avenir pour l'Irak.

--- Exemple train 1 ---
EN      : It does indeed speak for itself that those who are around the negotiating table are most sensitive to their own issues; this is always the case.
FR gold : Il est d'ailleurs évident que les 

In [23]:
# Quelques phrases du dataset validation
for i in range(3):
    en = val_ds[i]["translation"]["en"]
    fr_gold = val_ds[i]["translation"]["fr"]

    fr_base = translate_sentence(en, base_model, tokenizer)
    fr_lora = translate_sentence(en, lora_model, tokenizer)  # modèle fine-tuné LoRA

    print("--- Exemple val", i, "---")
    print("EN      :", en)
    print("FR gold :", fr_gold)
    print("FR base :", fr_base)
    print("FR LoRA :", fr_lora)
    print()

--- Exemple val 0 ---
EN      : On behalf of my group, I would encourage them to keep to this rule.
FR gold : Au nom de mon groupe, je l’encouragerai à poursuivre dans cette voie.
FR base : Au nom de mon groupe, je les encourage à respecter cette règle.
FR LoRA : Au nom de mon groupe, je les encourage à respecter cette règle.

--- Exemple val 1 ---
EN      : Madam President, unemployment is the European Union's number one problem.
FR gold : Madame le Président, dans l'Union européenne le chômage est le problème central.
FR base : Madame le Président, le chômage est le principal problème de l'Union européenne.
FR LoRA : Madame le Président, le chômage est le problème numéro un de l'Union européenne.

--- Exemple val 2 ---
EN      : The time when political movements, pressure groups and governments denied the existence of environmental problems is over.
FR gold : . (NL) Le temps où les courants politiques, les groupements d'intérêts et les autorités niaient l'existence des problèmes écol

In [24]:
# phrases de test perso
tests = [
    "Hello, how are you?",
    "This project is about automatic translation.",
    "The weather is nice today.",
]

for s in tests:
    print("EN      :", s)
    print("FR base :", translate_sentence(s, base_model, tokenizer))
    print("FR LoRA :", translate_sentence(s, lora_model, tokenizer))
    print()


EN      : Hello, how are you?
FR base : Bonjour, comment êtes-vous?
FR LoRA : Bonjour, comment êtes-vous?

EN      : This project is about automatic translation.
FR base : Ce projet concerne la traduction automatique.
FR LoRA : Ce projet concerne la traduction automatique.

EN      : The weather is nice today.
FR base : Le temps est agréable aujourd'hui.
FR LoRA : Le temps est agréable aujourd'hui.

